In [1]:
import sys, os, importlib
import rasterio

import geopandas as gpd
import pandas as pd

# Import GOST libraries; sys.path.append will be unnecessary if libraries are already installed
sys.path.append("../../../../gostrocks/src")

import GOSTRocks.rasterMisc as rMisc

/home/wb411133/.conda/envs/geog/lib/python3.7/site-packages/geopandas/_compat.py:88: UserWarning: The Shapely GEOS version (3.7.1-CAPI-1.11.1 0) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
# define and extract focal data
iso3 = "TUR"
nuts2_file = "/home/public/Data/COUNTRY/TUR/attributes/attributes.geojson"
inD = gpd.read_file(nuts2_file)
pop_file = "/home/public/Data/COUNTRY/TUR/Population/ppp_2020_1km_Aggregated.tif"

global_friction = "/home/public/Data/GLOBAL/INFRA/FRICTION_2020/2020_motorized_friction_surface.geotiff"
global_population = "/home/public/Data/GLOBAL/Population/WorldPop_PPP_2020/ppp_2020_1km_Aggregated.tif"
if not os.path.exists(pop_file):
    outFile = os.path.join("/home/wb411133/temp/TUR", os.path.basename(pop_file))
    rMisc.clipRaster(rasterio.open(global_population), inD, outFile)

urban_raster = "/home/public/Data/COUNTRY/TUR/URBAN_DATA/tur_urban.tif"
tt_folder = "/home/public/Data/COUNTRY/TUR/travel_time"
tt_airports = os.path.join(tt_folder, "least_cost_travel_time_airports.tif")
tt_hospitals = os.path.join(tt_folder, "least_cost_travel_time_hospitals.tif")

In [ ]:
# Standardize rasters to the population layer
popR = rasterio.open(pop_file)
for rFile in [urban_raster, tt_airports, tt_hospitals]:
    curR = rasterio.open(rFile)
    out_file = os.path.join("/home/wb411133/temp/TUR", os.path.basename(rFile))
    rMisc.standardizeInputRasters(rasterio.open(rFile), popR, out_file)


In [9]:
# summarize urbanization
popR = rasterio.open(pop_file)
res = rMisc.zonalStats(inD, popR, minVal=0)
res = pd.DataFrame(res, columns=['SUM', 'MIN', 'MAX', 'MEAN'])
inD['wp_pop'] = res['SUM']

urbanR = rasterio.open(urban_raster)
urbanD = urbanR.read()
popD = popR.read()
urbanPop = urbanD * popD

with rMisc.create_rasterio_inmemory(popR.profile, urbanPop) as urbanPopR:
    res = rMisc.zonalStats(inD, urbanPopR, minVal=0)
    res = pd.DataFrame(res, columns=['SUM', 'MIN', 'MAX', 'MEAN'])
    inD['urban_pop'] = res['SUM']
    
inD['per_urban'] = inD['urban_pop'] / inD['wp_pop']

In [7]:
#Summarize travel time
thresh = 60
popD = popR.read()

for tt_file in [tt_airports, tt_hospitals]:
    field_name = "%s_%s" % (tt_file.split("_")[-1].replace(".tif", ""), thresh)
    ttR = rasterio.open(tt_file)
    ttD = ttR.read() < thresh
    ttPop = popD * ttD
    with rMisc.create_rasterio_inmemory(popR.profile, ttPop) as urbanPopR:
        res = rMisc.zonalStats(inD, urbanPopR, minVal=0)
        res = pd.DataFrame(res, columns=['SUM', 'MIN', 'MAX', 'MEAN'])
        inD[field_name] = res['SUM']

In [10]:
inD.head()

,id,GID_0,NAME_0,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,nuts2_regions,nuts1_regions,nuts1_labels,...,nuts2_development_level_2004,nuts2_development_level_2018,nuts1_growth_speed,nuts2_growth_speed,geometry,wp_pop,urban_pop,airports_60,hospitals_60,per_urban
0,1,TUR,Turkey,,Il,Province,,TR10,TR1,Istanbul Region,...,More developed,More developed,Average growth,Average growth,"MULTIPOLYGON (((28.18166 41.56406, 28.18987 41...",1.578518e+07,1.422525e+07,1.514880e+07,1.532639e+07,0.901178
1,2,TUR,Turkey,,Il,Province,,TR21,TR2,West Marmara Region,...,More developed,More developed,Low growth,Low growth,"POLYGON ((28.03551 41.98308, 28.02805 41.98082...",1.755714e+06,9.234421e+05,6.146452e+05,1.704351e+06,0.525964
2,3,TUR,Turkey,,Il,Province,,TR22,TR2,West Marmara Region,...,Transition,Transition,Low growth,Low growth,"MULTIPOLYGON (((28.16478 40.39561, 28.16460 40...",1.719822e+06,8.313265e+05,3.985374e+05,1.532288e+06,0.483379
3,4,TUR,Turkey,,Il,Province,,TR31,TR3,Aegean Region,...,More developed,More developed,Average growth,Low growth,"MULTIPOLYGON (((28.44261 38.10466, 28.41953 38...",4.169740e+06,3.330121e+06,3.542556e+06,3.940764e+06,0.798640
4,5,TUR,Turkey,,Il,Province,,TR32,TR3,Aegean Region,...,Transition,Transition,Average growth,Low growth,"MULTIPOLYGON (((29.85212 37.75229, 29.80355 37...",3.033170e+06,1.715371e+06,8.557799e+05,2.660810e+06,0.565537


In [11]:
inD.to_file("/home/wb411133/temp/TUR/NUTS2_attributed.shp")